# Sección 2. Entendimiento y Preparación de los datos

Esta sección tiene como propósito exponer el entendimiento y la preparación de los textos para posteriormente utilizarlos en la implementación de los modelos de aprendizaje. Para la manipulación de los datos se usarán librerias como **pandas** y **numpy**. Para la visualización se utilizarán **matplotlib**, **seaborn** y **WordCloud**.

## 1. Instalación e importación de librerias

### Instalación

In [1]:
#%pip install pandas numpy matplotlib seaborn scikit-learn nltk spacy wordcloud tqdm
#%pip install spacy-langdetect
#!python -m spacy download es_core_news_sm


In [2]:
# Manipulación de datos
import pandas as pd
import numpy as np

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

# Procesamiento de texto
import re
import string
import nltk
import spacy
from nltk.corpus import stopwords
from spacy_langdetect import LanguageDetector
import re
import unicodedata
import nltk
from nltk.corpus import stopwords

# Modelado y evaluación
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Progreso en bucles
from tqdm import tqdm

# Cargar modelo en español de spaCy
nlp = spacy.load("es_core_news_sm")

# Descargar stopwords de NLTK en español
nltk.download('stopwords')
stopwords_es = set(stopwords.words('spanish'))

# Configuración de visualización
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (10,5)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\santi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# 2. Perfilamiento y entendimiento de los datos

### Carga de datos

In [4]:
# Cargar datos
data=pd.read_csv('fake_news_spanish.csv', sep=';', encoding = "utf-8")

news_df = data.copy()

In [5]:
news_df.head()

,ID,Label,Titulo,Descripcion,Fecha
0,ID,1,'The Guardian' va con Sánchez: 'Europa necesit...,El diario británico publicó este pasado jueves...,02/06/2023
1,ID,0,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,01/10/2023
2,ID,1,El 'Ahora o nunca' de Joan Fuster sobre el est...,El valencianismo convoca en Castelló su fiesta...,25/04/2022
3,ID,1,"Iglesias alienta a Yolanda Díaz, ERC y EH Bild...","En política, igual que hay que negociar con lo...",03/01/2022
4,ID,0,Puigdemont: 'No sería ninguna tragedia una rep...,"En una entrevista en El Punt Avui, el líder de...",09/03/2018


In [6]:
news_df.info()  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57063 entries, 0 to 57062
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID           57063 non-null  object
 1   Label        57063 non-null  int64 
 2   Titulo       57047 non-null  object
 3   Descripcion  57063 non-null  object
 4   Fecha        57063 non-null  object
dtypes: int64(1), object(4)
memory usage: 2.2+ MB


# 3. Preparación de los datos

### 3.1. Limpieza de datos.

In [7]:
def remove_non_ascii(words):
    """Remueve caracteres no ASCII de una lista de palabras"""
    new_words = []
    for word in words:
        if word is not None:
            new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
            new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convierte todas las palabras a minúsculas"""
    return [word.lower() for word in words if word is not None]

def remove_punctuation(words):
    """Elimina signos de puntuación"""
    return [re.sub(r'[^\w\s]', '', word) for word in words if word is not None and re.sub(r'[^\w\s]', '', word) != '']

def remove_numbers(words):
    """Elimina los números de la lista de palabras"""
    return [word for word in words if not word.isdigit()]

def remove_stopwords(words):
    """Elimina stopwords en español"""
    return [word for word in words if word not in stopwords_es]

def preprocessing(text):
    """Aplica todas las funciones de limpieza de texto a un string"""
    words = text.split()  # Tokenización básica separando por espacios
    words = to_lowercase(words)
    words = remove_non_ascii(words)
    words = remove_punctuation(words)
    words = remove_numbers(words)
    words = remove_stopwords(words)
    return " ".join(words)  # Retorna el texto limpio como un string


In [8]:
# Aplicar la limpieza a la columna 'Titulo' y 'Descripcion'
news_df['Titulo_Limpio'] = news_df['Titulo'].astype(str).apply(preprocessing)
news_df['Descripcion_Limpia'] = news_df['Descripcion'].astype(str).apply(preprocessing)

# Compración antes y después de la limpieza
print(news_df[['Titulo', 'Titulo_Limpio']].head())
print(news_df[['Descripcion', 'Descripcion_Limpia']].head())


                                              Titulo  \
0  'The Guardian' va con Sánchez: 'Europa necesit...   
1  REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...   
2  El 'Ahora o nunca' de Joan Fuster sobre el est...   
3  Iglesias alienta a Yolanda Díaz, ERC y EH Bild...   
4  Puigdemont: 'No sería ninguna tragedia una rep...   

                                       Titulo_Limpio  
0  the guardian va sanchez europa necesita apuest...  
1  revelan gobierno negocio liberacion mireles ca...  
2  ahora nunca joan fuster estatuto valenciano cu...  
3  iglesias alienta yolanda diaz erc eh bildu neg...  
4  puigdemont seria ninguna tragedia repeticion e...  
                                         Descripcion  \
0  El diario británico publicó este pasado jueves...   
1  REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...   
2  El valencianismo convoca en Castelló su fiesta...   
3  En política, igual que hay que negociar con lo...   
4  En una entrevista en El Punt Avui, el líder de... 

### 3.2. Tokenización

In [ ]:
nltk.download('punkt')  # Descarga el tokenizer de NLTK


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\santi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [10]:
from nltk.tokenize import word_tokenize
nltk.download('punkt_tab')  # Descarga el tokenizer de NLTK
def tokenize_nltk(text):
    """Tokeniza el texto usando NLTK"""
    return word_tokenize(text, language="spanish")  # Especificamos español

# Aplicar tokenización en el DataFrame
news_df['Titulo_Tokens'] = news_df['Titulo'].astype(str).apply(tokenize_nltk)
news_df['Descripcion_Tokens'] = news_df['Descripcion'].astype(str).apply(tokenize_nltk)

# Mostrar ejemplos
print(news_df[['Titulo', 'Titulo_Tokens']].head())


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\santi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


                                              Titulo  \
0  'The Guardian' va con Sánchez: 'Europa necesit...   
1  REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...   
2  El 'Ahora o nunca' de Joan Fuster sobre el est...   
3  Iglesias alienta a Yolanda Díaz, ERC y EH Bild...   
4  Puigdemont: 'No sería ninguna tragedia una rep...   

                                       Titulo_Tokens  
0  ['The, Guardian, ', va, con, Sánchez, :, 'Euro...  
1  [REVELAN, QUE, EL, GOBIERNO, NEGOCIO, LA, LIBE...  
2  [El, 'Ahora, o, nunca, ', de, Joan, Fuster, so...  
3  [Iglesias, alienta, a, Yolanda, Díaz, ,, ERC, ...  
4  [Puigdemont, :, 'No, sería, ninguna, tragedia,...  


### 3.3. Normalización